In [ ]:
using JuMP
using MadNLP
using ProgressMeter
using CairoMakie
import Plots.plot as pplot
import Plots.plot! as pplot!

In [ ]:
# Lennard-jones of squared distances instead of distances
function lennard_jones(cur_dist_sq; pref_dist=1, pot_min=-1)
    # the factor 2 assures that the global
    # minimum is attained at cur_dist == pref_dist
    q = pref_dist^2/(2*cur_dist_sq)
    -4*pot_min*(q^2 - q)
end

In [ ]:
N_h = 10
triangle_side_length = 1/(N_h-1)
triangle_height = sqrt(3)/2*triangle_side_length
N_v = floor(Int, 1 / triangle_height) + 1
max_stretch = 0.8
time_stretch = 3
max_pull = 0.4
max_push = 0.3
time_pull_and_push = 2
time_pause = 0.
fps = 120
pref_dist = 1/(N_h-1)
break_dist = 2*pref_dist
pot_min = -1.
search_rad = 1.
diss_coeff = 1.
alpha = 100
eps = pref_dist/100
animation_width = 800
hidpi_scaling = 4
file_name = "square_pull"
two_sided_pull = false
fontsize = 16
only_video = true

In [ ]:
function dirichlet_delta_pull(t; max_stretch=1, time_stretch=1, time_pause=1.5)
    if t <= time_stretch
        return max_stretch*t/time_stretch
    else
        return max_stretch
    end
end

function dirichlet_delta_pull_and_push(t; max_pull=1, max_push=1, time_pull_and_push=1.5, time_pause=0.25)
    time_pull = time_pull_and_push/2
    time_push = time_pull
    if t <= time_pull
        return max_pull*t/time_pull
    elseif t <= time_pull + time_pause
        return max_pull
    elseif t <= time_pull_and_push + time_pause
        return max_pull - max_push*(t - time_pull - time_pause)/time_push
    else
        return max_pull - max_push      
    end
end

# dirichlet_delta(t) = dirichlet_delta_pull_and_push(
#     t;
#     max_pull=max_pull,
#     max_push=max_push,
#     time_pull_and_push=time_pull_and_push, 
#     time_pause=time_pause
# )
dirichlet_delta(t) = dirichlet_delta_pull(
    t;
    max_stretch=max_stretch,
    time_stretch=time_stretch, 
    time_pause=time_pause
)

g_right(t) = 1 + triangle_side_length/2 + dirichlet_delta(t)
if two_sided_pull
    g_left(t) = -dirichlet_delta(t)
else
    g_left(t) = 0
end

time_horizon = time_stretch + time_pause
xs = range(0, time_horizon, length=1000)
ys = g_right.(xs)
ymin = two_sided_pull ? -max_stretch : 0
pplot(xs, g_right.(xs), ylims=(ymin -.1, 1 + max_stretch + .1), label="g_right")
pplot!(xs, g_left.(xs), ylims=(ymin -.1, 1 + max_stretch + .1), label="g_left")

In [ ]:
soft_max(x, y) = (x*exp(alpha*x) + y*exp(alpha*y)) / 
    (exp(alpha*x) + exp(alpha*y))
xs = range(-2., 2., length=1000)
soft_ys = [soft_max(x, 0.) for x in xs]
ys = [max(x, 0.) for x in xs]
pplot(xs, soft_ys, label="soft_+", legend=:topleft)
pplot!(xs, ys, label="+")

In [ ]:
function initialize_jump_on_m1()
    step = 2
    minmove_l2 = Model(MadNLP.Optimizer)

    @NLparameter(minmove_l2, prev_y[i=1:N] == (i-1)/(N-1))

    @variable(
        minmove_l2,
        value(prev_y[i]) - search_rad <= y[i=1:N] <= value(prev_y[i]) + search_rad,
        start=value(prev_y[i])
    )


    @constraint(
        minmove_l2, dirichlet_right,
        y[N] == g_right((step-1)/fps)
    )
    @constraint(
        minmove_l2, dirichlet_left,
        y[1] == g_left((step-1)/fps)
    )

    @expression(minmove_l2, dist_sq[i=1:N-1], (y[i+1] - y[i])^2)
    @NLexpression(
        minmove_l2, energy, 
        .5/(N-1)*sum(dist_sq[i] for i in 1:N-1)
    )

    @NLexpression(
        minmove_l2, dissipation,
        .5/N*sum((y[i] - prev_y[i])^2 for i in 2:N-1)
    )
    @NLobjective(minmove_l2, Min, energy + diss_coeff*fps*dissipation)
end

# initialize_jump_on_m1()

In [ ]:
step = 2

# TODO: add constraint to prevent triangle cliping
# TODO: reimplement using vertices, edges, triangles

minmove_l2 = Model(
    optimizer_with_attributes(
        MadNLP.Optimizer,
        "print_level" => MadNLP.WARN,
        "blas_num_threads" => 8
    )
)

# i -> row, j -> column, start top left
# i, j indix vertices
@NLparameter(
    minmove_l2, 
    prev_x[i=1:N_v, j=1:N_h] == (i-1) % 2 * triangle_side_length/2 + (j-1)*triangle_side_length
)
@NLparameter(
    minmove_l2,
    prev_y[i=1:N_v, j=1:N_h] == 1 - (i-1)*triangle_height
)

# track maximal elongation (i, j index triangles)
N_base_v = N_v
N_base_h = N_h-1
N_zig_odd_v = floor(Int, N_v/2)
N_zig_odd_h = N_h-1
N_zig_even_v = floor(Int, (N_v-1)/2)
N_zig_even_h = N_h
N_zag_odd_v = N_zig_odd_v
N_zag_odd_h = N_h
N_zag_even_v = N_zig_even_v
N_zag_even_h = N_h-1

@NLparameter(
    minmove_l2,
    max_elong_sq_base[i=1:N_base_v, j=1:N_base_h] == triangle_side_length^2
)
@NLparameter(
    minmove_l2,
    max_elong_sq_zig_odd[i=1:N_zig_odd_v, j=1:N_zig_odd_h] == triangle_side_length^2
)
@NLparameter(
    minmove_l2,
    max_elong_sq_zig_even[i=1:N_zig_even_v, j=1:N_zig_even_h] == triangle_side_length^2
)
@NLparameter(
    minmove_l2,
    max_elong_sq_zag_odd[i=1:N_zag_odd_v, j=1:N_zag_odd_h] == triangle_side_length^2
)
@NLparameter(
    minmove_l2,
    max_elong_sq_zag_even[i=1:N_zag_even_v, j=1:N_zag_even_h] == triangle_side_length^2
)

# track broken links
@NLparameter(
    minmove_l2,
    broken_base[i=1:N_base_v, j=1:N_base_h] == 0.
)
@NLparameter(
    minmove_l2,
    broken_zig_odd[i=1:N_zig_odd_v, j=1:N_zig_odd_h] == 0.
)
@NLparameter(
    minmove_l2,
    broken_zig_even[i=1:N_zig_even_v, j=1:N_zig_even_h] == 0.
)
@NLparameter(
    minmove_l2,
    broken_zag_odd[i=1:N_zag_odd_v, j=1:N_zag_odd_h] == 0.
)
@NLparameter(
    minmove_l2,
    broken_zag_even[i=1:N_zag_even_v, j=1:N_zag_even_h] == 0.
)

# variables
@variable(
    minmove_l2,
    value(prev_x[i,j]) - search_rad <= x[i=1:N_v, j=1:N_h] <= value(prev_x[i,j]) + search_rad,
    start=value(prev_x[i,j])
)
@variable(
    minmove_l2,
    value(prev_y[i,j]) - search_rad <= y[i=1:N_v, j=1:N_h] <= value(prev_y[i,j]) + search_rad,
    start=value(prev_y[i,j])
)

# Dirichlet condition
# TODO: prescribe boundary condition on ALL points on the left and right
N_dirichlet_left = floor(Int, (N_v+1)/2)
@constraint(
    minmove_l2, dirichlet_left_x[i=1:N_dirichlet_left],
    x[1+2(i-1),1] == g_left((step-1)/fps)
)
@constraint(
    minmove_l2, dirichlet_left_y[i=1:N_dirichlet_left],
    y[1+2(i-1),1] == 1 - 2(i-1)*triangle_height
)
N_dirichlet_right = floor(Int, N_v/2)
@constraint(
    minmove_l2, dirichlet_right_x[i=1:N_dirichlet_right],
    x[2i,end] == g_right((step-1)/fps)
)

@constraint(
    minmove_l2, dirichlet_right_y[i=1:N_dirichlet_right],
    y[2i,end] == 1 - (2i-1)*triangle_height
)

# disallow clipping of triangles
N_triang_v = N_v - 1
N_triang_h = 2*(N_h - 1)
# TODO

# energy
# quared distances
@expression(
    minmove_l2, dist_sq_base[i=1:N_base_v, j=1:N_base_h],
    (x[i,j+1] - x[i, j])^2 + (y[i,j+1] - y[i, j])^2
)
@expression(
    minmove_l2, dist_sq_zig_odd[i=1:N_zig_odd_v, j=1:N_zig_odd_h],
    (x[2i-1,j+1] - x[2i, j])^2 + (y[2i-1,j+1] - y[2i, j])^2
)
@expression(
    minmove_l2, dist_sq_zig_even[i=1:N_zig_even_v, j=1:N_zig_even_h],
    (x[2i,j] - x[1+2i, j])^2 + (y[2i,j] - y[1+2i, j])^2
)
@expression(
    minmove_l2, dist_sq_zag_odd[i=1:N_zag_odd_v, j=1:N_zag_odd_h],
    (x[2i,j] - x[1+2(i-1), j])^2 + (y[2i,j] - y[1+2(i-1), j])^2
)
@expression(
    minmove_l2, dist_sq_zag_even[i=1:N_zag_even_v, j=1:N_zag_even_h],
    (x[1+2i,j+1] - x[2i, j])^2 + (y[1+2i,j+1] - y[2i, j])^2
)
# damaged distances
@NLexpression(
    minmove_l2, damaged_dist_sq_base[i=1:N_base_v, j=1:N_base_h],
    ( 
        dist_sq_base[i,j]*exp(alpha*dist_sq_base[i,j]) 
        + broken_base[i,j]*max_elong_sq_base[i,j]*exp(alpha*broken_base[i,j]*max_elong_sq_base[i,j])
    ) / (exp(alpha*dist_sq_base[i,j]) + exp(alpha*broken_base[i,j]*max_elong_sq_base[i,j]))
)
@NLexpression(
    minmove_l2, damaged_dist_sq_zig_odd[i=1:N_zig_odd_v, j=1:N_zig_odd_h],
    ( 
        dist_sq_zig_odd[i,j]*exp(alpha*dist_sq_zig_odd[i,j]) 
        + broken_zig_odd[i,j]*max_elong_sq_zig_odd[i,j]*exp(alpha*broken_zig_odd[i,j]*max_elong_sq_zig_odd[i,j])
    ) / (exp(alpha*dist_sq_zig_odd[i,j]) + exp(alpha*broken_zig_odd[i,j]*max_elong_sq_zig_odd[i,j]))
)
@NLexpression(
    minmove_l2, damaged_dist_sq_zig_even[i=1:N_zig_even_v, j=1:N_zig_even_h],
    ( 
        dist_sq_zig_even[i,j]*exp(alpha*dist_sq_zig_even[i,j]) 
        + broken_zig_even[i,j]*max_elong_sq_zig_even[i,j]*exp(alpha*broken_zig_even[i,j]*max_elong_sq_zig_even[i,j])
    ) / (exp(alpha*dist_sq_zig_even[i,j]) + exp(alpha*broken_zig_even[i,j]*max_elong_sq_zig_even[i,j]))
)
@NLexpression(
    minmove_l2, damaged_dist_sq_zag_odd[i=1:N_zag_odd_v, j=1:N_zag_odd_h],
    ( 
        dist_sq_zag_odd[i,j]*exp(alpha*dist_sq_zag_odd[i,j]) 
        + broken_zag_odd[i,j]*max_elong_sq_zag_odd[i,j]*exp(alpha*broken_zag_odd[i,j]*max_elong_sq_zag_odd[i,j])
    ) / (exp(alpha*dist_sq_zag_odd[i,j]) + exp(alpha*broken_zag_odd[i,j]*max_elong_sq_zag_odd[i,j]))
)
@NLexpression(
    minmove_l2, damaged_dist_sq_zag_even[i=1:N_zag_even_v, j=1:N_zag_even_h],
    ( 
        dist_sq_zag_even[i,j]*exp(alpha*dist_sq_zag_even[i,j]) 
        + broken_zag_even[i,j]*max_elong_sq_zag_even[i,j]*exp(alpha*broken_zag_even[i,j]*max_elong_sq_zag_even[i,j])
    ) / (exp(alpha*dist_sq_zag_even[i,j]) + exp(alpha*broken_zag_even[i,j]*max_elong_sq_zag_even[i,j]))
)

# plug into Lennard-Jones and sum up
W(d) = lennard_jones(d, pref_dist=pref_dist, pot_min=pot_min)
register(minmove_l2, :W, 1, W, autodiff = true)
@NLexpression(
    minmove_l2, energy_base,
    sum(W(damaged_dist_sq_base[i,j]) for i in 1:N_base_v, j in 1:N_base_h)
)
@NLexpression(
    minmove_l2, energy_zig_odd,
    sum(W(damaged_dist_sq_zig_odd[i,j]) for i in 1:N_zig_odd_v, j in 1:N_zig_odd_h)
)
@NLexpression(
    minmove_l2, energy_zig_even,
    sum(W(damaged_dist_sq_zig_even[i,j]) for i in 1:N_zig_even_v, j in 1:N_zig_even_h)
)
@NLexpression(
    minmove_l2, energy_zag_odd,
    sum(W(damaged_dist_sq_zag_odd[i,j]) for i in 1:N_zag_odd_v, j in 1:N_zag_odd_h)
)
@NLexpression(
    minmove_l2, energy_zag_even,
    sum(W(damaged_dist_sq_zag_even[i,j]) for i in 1:N_zag_even_v, j in 1:N_zag_even_h)
)
N_links = N_base_v*N_base_h +
    N_zig_odd_v*N_zig_odd_h + 
    N_zig_even_v*N_zig_even_h + 
    N_zag_odd_v*N_zag_odd_h + 
    N_zag_even_v*N_zag_even_h
@NLexpression(
    minmove_l2, energy,
    1/N_links*(energy_base + energy_zig_odd + energy_zig_even + energy_zag_odd + energy_zag_even)
)

# dissipation
@NLexpression(
    minmove_l2, dissipation_odd,
    sum((x[i,j] - prev_x[i,j])^2 + (y[i,j] - prev_y[i,j])^2 for i in 1:2:N_v, j in 2:N_h)
)
@NLexpression(
    minmove_l2, dissipation_even,
    sum((x[i,j] - prev_x[i,j])^2 + (y[i,j] - prev_y[i,j])^2 for i in 2:2:N_v, j in 1:N_h-1)
)
N_free_atoms = N_v * N_h - N_dirichlet_left - N_dirichlet_right
@NLexpression(
    minmove_l2, dissipation,
    0.5/N_free_atoms*(dissipation_odd + dissipation_even)
)

@NLobjective(minmove_l2, Min, energy + diss_coeff*fps*dissipation)

minmove_l2

In [ ]:
function interleave(first, second)
    # currently only works for vectors of the same size
    N = size(first)[1]
    interleaved = similar(first, 2N)
    for i in 1:N
        interleaved[2*(i-1)+1] = first[i]
        interleaved[2*i] = second[i]
    end
    interleaved
end

a = [i for i in 1:5]
b = [i+5 for i in 1:5]
a, b, interleave(a, b)

In [ ]:
function _plot_step!(
        ax, x, y,
        scale,
        atom_color, bond_color, broken_bond_color
)
    # broken_bond_color currently ignored (TODO)
    empty!(ax)
    
    N_v, N_h = size(x)
    
    # draw base lines
    for i in 1:N_v
        lines!(x[i,:], y[i,:], color=bond_color, linewidth=scale*1.5, linestyle=:dash)
    end
    # draw zag-zigs
    for i in 1:2:N_v-1
        # interleave coordinates
        xs = interleave(x[i, :], x[i+1, :])
        ys = interleave(y[i, :], y[i+1, :])
        lines!(xs, ys, color=bond_color, linewidth=scale*1.5, linestyle=:dash)
    end
    # draw zig-zags
    for i in 2:2:N_v-1
        # interleave coordinates
        xs = interleave(x[i+1, :], x[i, :])
        ys = interleave(y[i+1, :], y[i, :])
        lines!(xs, ys, color=bond_color, linewidth=scale*1.5, linestyle=:dash)
    end
    
    
    # draw vertices
    scatter!([x...], [y...], color=atom_color, markersize=scale*6)
end

function plot_step(
        x, y;
        width=800, scale=1,
        atom_color=:blue, bond_color=:green, broken_bond_color=:red
)
    height = width
    N_v, N_h = size(x)
    left = minimum([x[i, 1] for i in 1:N_v])
    right = maximum([x[i, end] for i in 1:N_v])
    bottom = minimum([y[end, j] for j in 1:N_h])
    up = maximum([y[1, j] for j in 1:N_h])
    delta = 0.05
    
    fig = Figure(resolution=(scale*width, scale*height), fontsize=scale*fontsize)
    ax = Axis(
        fig[1, 1],
        limits=(left-delta, right+delta, bottom-delta, up+delta),
        aspect=1,
    )
    
    _plot_step!(ax, x, y, scale, atom_color, bond_color, broken_bond_color)
    
    fig, ax # remove
end

fig, ax = plot_step(value.(prev_x), value.(prev_y), scale=hidpi_scaling, width=animation_width)
fig

In [ ]:
function mark_triangle!(i, j)
    
end

fig

In [ ]:
optimize!(minmove_l2)

In [ ]:
plot_step(value.(x), value.(y), scale=hidpi_scaling, width=animation_width)

In [ ]:
minmoves = [(value.(prev_x), value.(prev_y)), (value.(x), value.(y))]
max_elong_sq = [max_elong_sq_base, max_elong_sq_zig_odd, max_elong_sq_zig_even, max_elong_sq_zag_odd, max_elong_sq_zag_even]
broken = [broken_base, broken_zig_odd, broken_zig_even, broken_zag_odd, broken_zag_even];

In [ ]:
# @showprogress "Computing minmoves..." for step in 3:fps*time_horizon
dissval = value(dissipation)
for step in 3:floor(Int, fps*time_horizon)
    xval = value.(x)
    yval = value.(y)

    # Compute new distances
    new_dist_sq_base = [(xval[i,j+1] - xval[i, j])^2 + (yval[i,j+1] - yval[i, j])^2 for i=1:N_base_v, j=1:N_base_h]
    new_dist_sq_zig_odd = [(xval[2i-1,j+1] - xval[2i, j])^2 + (yval[2i-1,j+1] - yval[2i, j])^2 for i=1:N_zig_odd_v, j=1:N_zig_odd_h]
    new_dist_sq_zig_even = [(xval[2i,j] - xval[1+2i, j])^2 + (yval[2i,j] - yval[1+2i, j])^2 for i=1:N_zig_even_v, j=1:N_zig_even_h]
    new_dist_sq_zag_odd = [(xval[2i,j] - xval[1+2(i-1), j])^2 + (yval[2i,j] - yval[1+2(i-1), j])^2 for i=1:N_zag_odd_v, j=1:N_zag_odd_h]
    new_dist_sq_zag_even = [(xval[1+2i,j+1] - xval[2i, j])^2 + (yval[1+2i,j+1] - yval[2i, j])^2 for i=1:N_zag_even_v, j=1:N_zag_even_h]
    new_dist_sq = [new_dist_sq_base, new_dist_sq_zig_odd, new_dist_sq_zig_even, new_dist_sq_zag_odd, new_dist_sq_zag_even]

    for (old_max_elong_sq, comp_max_elong_sq, old_broken) in zip(max_elong_sq, new_dist_sq, broken)
        # update maximal elongation
        new_max_elong_sq = max.(value.(old_max_elong_sq), comp_max_elong_sq)
        set_value.(old_max_elong_sq, new_max_elong_sq)

        # check for newly broken links
        new_broken = [max_dist > break_dist ? 1. : 0. for max_dist in new_max_elong_sq]
        set_value.(old_broken, new_broken)
    end

    # update atom locations
    set_value.(prev_x, value.(x))
    set_value.(prev_y, value.(y))

    # update boundary values
    # we only need to update the x-coordinates
    new_left_x = g_left((step-1)/fps)
    new_dirichlet_left_x = [new_left_x for _ in 1:N_dirichlet_left]
    set_normalized_rhs.(dirichlet_left_x, new_dirichlet_left_x)
    new_right_x = g_right((step-1)/fps)
    new_dirichlet_right_x = [new_right_x for _ in 1:N_dirichlet_right]
    set_normalized_rhs.(dirichlet_right_x, new_dirichlet_right_x)

    optimize!(minmove_l2)

    push!(minmoves, (value.(x), value.(y)))
    
    dissval_new = value(dissipation)
    xval, _ = minmoves[end]
    println("$step: $(xval[2, 10]), $new_right_x, $(dissval_new/dissval)")
    if dissval_new/dissval > 1000
        break
    end
    dissval = dissval_new
end

In [ ]:
plot_step(minmoves[end-1]..., scale=hidpi_scaling, width=animation_width)

In [ ]:
plot_step(minmoves[end]..., scale=hidpi_scaling, width=animation_width)

In [ ]:
# function animate_steps(minmoves, file_name; fps=30, width=600, fontsize=11, scale=1, save_snapshots=false)
#     experiments_folder = "experiments"
#     if !ispath(experiments_folder)
#         mkdir(experiments_folder)
#     end
#     if save_snapshots
#         snapshots_folder = "$experiments_folder/$file_name"
#         if !ispath(snapshots_folder)
#             mkdir(snapshots_folder)
#         end
#     end
    
#     aspect = 4
#     ax_aspect = 4
#     height = width/aspect
#     max_right = maximum([y[end] for y in minmoves])
#     min_left = minimum([y[1] for y in minmoves])
#     delta = 0.1
#     max_len = max_right - min_left + 2delta
#     ax_height = max_len/ax_aspect
#     N = length(minmoves[1])
    
#     fig = Figure(resolution=(scale*width, scale*height), fontsize=scale*fontsize)
#     ax = Axis(
#         fig[1, 1],
#         limits=(min_left-delta, max_right+delta, -ax_height/2, ax_height/2),
#         aspect=ax_aspect,
#     )
    
#     y = zeros(N)
    
#     frame = 1
#     record(fig, "experiments/$file_name.mp4", minmoves; framerate = fps) do (x)
#         empty!(ax)
#         scatter!(x, y, color=:blue, markersize=scale*6)
#         if save_snapshots
#             save("$snapshots_folder/$(file_name)_$frame.pdf", fig)
#         end
#         frame += 1
#     end
# end

In [ ]:
# animate_steps(
#     minmoves, file_name, 
#     fps=60, 
#     width=animation_width, 
#     fontsize=fontsize,
#     scale=hidpi_scaling,
#     save_snapshots=!only_video
# )

In [ ]:
# get_total(value.(y), value.(prev_y), fps, diss_coeff)